# Exercise 1: Performing SIFA and SEFA on simulation data

First we'll try to perform SIFA and SEFA on simulated data. This gives us the opportunity to test our implementation and to see how the algorithms work. In the next exercise we'll apply the algorithms to real data.

In this notebook you'll run into sections that are marked **Assignment**. These sections contain functions that you need to implement yourself. You can check your implementation by running the tests that are provided in the same section. When everything works, show it to one of the trainers and then continue with the next exercise.

In [ ]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import secrets
import ipytest

ipytest.autoconfig()

%load_ext autoreload
%autoreload 2

## Load example dataset

First we load an example dataset. This we can use to test our code while we implement our SIFA algorithm. Later we will use the same code to run the algorithm on a real dataset.

In [ ]:
# Import simulation data from .npz file
data = np.load('simulation_data.npz', allow_pickle=True)

# Extract data from .npz file
effective_ciphertexts = data['effective_ciphertexts']
ineffective_ciphertexts = data['ineffective_ciphertexts']
correct_key = data['key']

# Function to print summary of data
def summarize_data(name, data):
    print(f'\n{name} summary:')
    print(f'\tShape:  {data.shape}')
    print(f'\tSize:   {data.size}')
    print(f'\tUnique: {np.unique(data).size}')

    print('\n\tFirst 5 entries:')
    for d in data[:5]:
        print('\t ', d)

# Print summary of data
print("correct_key:", correct_key)
summarize_data('ineffective_ciphertexts', ineffective_ciphertexts)
summarize_data('effective_ciphertexts', effective_ciphertexts)

## Implementing AES encryption in Python

The low level methods defined in [`aes_lib.AESCore`](aes_lib.py) can be used to implement AES functions:

In [ ]:
from aes_lib import AESCore

def aes_encrypt(key, plaintext):
    # Create an AESCore object
    aes = AESCore()

    # Generate the subkeys
    key_matrices = aes.expand_key(key)

    # Encrypt the plaintext
    state=aes.bytes2matrix(plaintext)

    aes.add_round_key(state, key_matrices[0])

    for i in range(1, 10):
        aes.sub_bytes(state)
        aes.shift_rows(state)
        aes.mix_columns(state)
        aes.add_round_key(state, key_matrices[i])

    aes.sub_bytes(state)
    aes.shift_rows(state)
    aes.add_round_key(state, key_matrices[-1])

    return aes.matrix2bytes(state)

In [ ]:
%%ipytest

# Test the aes_encrypt function

def test_aes_encrypt_nist_vector1():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    plaintext = bytes.fromhex("6bc1bee22e409f96e93d7e117393172a")
    ciphertext = bytes.fromhex("3ad77bb40d7a3660a89ecaf32466ef97")
    assert aes_encrypt(key, plaintext) == ciphertext

def test_aes_encrypt_nist_vector2():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    plaintext = bytes.fromhex("ae2d8a571e03ac9c9eb76fac45af8e51")
    ciphertext = bytes.fromhex("f5d3d58503b9699de785895a96fdbaaf")
    assert aes_encrypt(key, plaintext) == ciphertext

def test_aes_encrypt_nist_vector3():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    plaintext = bytes.fromhex("30c81c46a35ce411e5fbc1191a0a52ef")
    ciphertext = bytes.fromhex("43b1cd7f598ece23881b00e3ed030688")
    assert aes_encrypt(key, plaintext) == ciphertext

The same goes for decryption:

In [ ]:
from aes_lib import AESCore

def aes_decrypt(key, ciphertext):
    # Create an AESCore object
    aes = AESCore()

    # Generate the subkeys
    key_matrices = aes.expand_key(key)

    # Decrypt the ciphertext
    state=aes.bytes2matrix(ciphertext)

    aes.add_round_key(state, key_matrices[-1])

    for i in range(9, 0, -1):
        aes.inv_shift_rows(state)
        aes.inv_sub_bytes(state)
        aes.add_round_key(state, key_matrices[i])
        aes.inv_mix_columns(state)

    aes.inv_shift_rows(state)
    aes.inv_sub_bytes(state)
    aes.add_round_key(state, key_matrices[0])

    return aes.matrix2bytes(state)

In [ ]:
%%ipytest

# Test the aes_decrypt function

def test_aes_decrypt_nist_vector1():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    plaintext = bytes.fromhex("6bc1bee22e409f96e93d7e117393172a")
    ciphertext = bytes.fromhex("3ad77bb40d7a3660a89ecaf32466ef97")
    assert aes_decrypt(key, ciphertext) == plaintext

def test_aes_decrypt_nist_vector2():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    plaintext = bytes.fromhex("ae2d8a571e03ac9c9eb76fac45af8e51")
    ciphertext = bytes.fromhex("f5d3d58503b9699de785895a96fdbaaf")
    assert aes_decrypt(key, ciphertext) == plaintext

def test_aes_decrypt_nist_vector3():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    plaintext = bytes.fromhex("30c81c46a35ce411e5fbc1191a0a52ef")
    ciphertext = bytes.fromhex("43b1cd7f598ece23881b00e3ed030688")
    assert aes_decrypt(key, ciphertext) == plaintext

## Exercise 1a: Decrypt to intermediate value

The first step is to calculate the intermediate value that was targeted by the fault for every ciphertext. As shown in [*"Fault Attacks on AES with Faulty Ciphertexts Only"* by Furh et al](https://www.ssi.gouv.fr/uploads/IMG/pdf/Fault_Attacks_on_AES_with_Faulty_Ciphertexts_Only.pdf). this attack can be mounted on $\tilde{S} m o d_9^i$ as the inverse SubBytes and the AddRoundKey operations only perform a permutation of the values of the end distribution, therefore keeping the number of occurrences as an invariant for every hypotheses. 

$
\tilde{S} m o d_9^i=M C^{-1} \circ S B^{-1} \circ S R^{-1}\left(\tilde{C}^i \oplus K_{10}\right) \text {. }
$

We can therefore calculate the intermediate values for every ciphertext by applying the inverse operations of the AES encryption algorithm. We can then use these intermediate values to calculate the probabilities of the key hypotheses.

### **Assignment**
**Implement an function `aes_decrypt_to_intermediate(key, ciphertext)` that takes a key and an ciphertext and calculate the $\tilde{S} m o d_9^i$ intermediate value.**


In [ ]:
from aes_lib import AESCore

# Remove the following line and implement the function below
raise NotImplementedError("Complete the assignment: implement the aes_decrypt_to_intermediate function")

def aes_decrypt_to_intermediate(key, ciphertext):
    aes = AESCore()
    state = aes.bytes2matrix(ciphertext)
    
    # 1. TODO Perform key expansion to get the subkeys
    # 2. TODO Perform add round key operation with the last subkey
    # 3. TODO Perform inverse shift rows operation
    # 4. TODO Perform inverse sub bytes operation
    # 5. TODO Perform inverse mix columns operation
    
    intermediate_value = aes.matrix2bytes(state)
    return intermediate_value

Now run the following code to test your implementation, if all three tests pass you can continue to the next step:

In [ ]:
%%ipytest

# Test aes_decrypt_to_intermediate function

def test_aes_decrypt_to_intermediate_nist_vector1():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    ciphertext = bytes.fromhex("3ad77bb40d7a3660a89ecaf32466ef97")
    intermediate_value = bytes.fromhex("8f48aaccec0c4c1371a83c9911427aea")
    assert aes_decrypt_to_intermediate(key, ciphertext) == intermediate_value

def test_aes_decrypt_to_intermediate_nist_vector2():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    ciphertext = bytes.fromhex("f5d3d58503b9699de785895a96fdbaaf")
    intermediate_value = bytes.fromhex("f2178e8abaf98be452174f77f3b99ce5")
    assert aes_decrypt_to_intermediate(key, ciphertext) == intermediate_value

def test_aes_decrypt_to_intermediate_nist_vector3():
    key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
    ciphertext = bytes.fromhex("43b1cd7f598ece23881b00e3ed030688")
    intermediate_value = bytes.fromhex("08b809e88801930d1aa7f3d1838973b9")
    assert aes_decrypt_to_intermediate(key, ciphertext) == intermediate_value

## Calculate intermediate values for all ciphertexts

Now we can calculate the intermediate values for all ciphertexts in the dataset. We can then use these intermediate values to calculate the probabilities of the key hypotheses.

In [ ]:
ineffective_intermediates = np.vectorize(aes_decrypt_to_intermediate)(correct_key, ineffective_ciphertexts)
effective_intermediates = np.vectorize(aes_decrypt_to_intermediate)(correct_key, effective_ciphertexts)

## Exercise 1b: Calculate bias for each byte

Next we calculate the bias for each byte of the intermediate values. For this the [`np.histogram`](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html) function can be used. It is important when using `np.histogram` to set the arguments `bins = 256` and `range=(0,256)`. This will ensure that the histogram is calculated for every possible byte value.

### **Assignment**
**Implement an function `calc_p_distribution(intermediates, normalize=True)` that takes an array of intermediates and boolean to determine of the output should be normalized.**

In [ ]:
# Remove the following line and implement the function below
raise NotImplementedError("Complete the assignment: implement the calc_p_distribution function")

def calc_p_distribution(intermediates, normalize=True):
    hists = np.zeros((16, 256))

    #    NOTE intermediates shape is (n,)
    # 1. TODO Convert each bytearray to an array of integers, shape should be (n, 16)
    # 2. TODO Transpose the array, shape should be (16, n)
    # 3. TODO Calculate the histogram of each row, shape should be (16, 256)
    # 4. TODO Normalize the histograms, shape should be (16, 256)

    return hists 

Now run the following code to test your implementation, if all three tests pass you can continue to the next step:

In [ ]:
%%ipytest

# Test calc_p_distribution function

def test_calc_p_distribution_1():
    data = np.load("test_data/p_distributions.npz")
    assert np.all(calc_p_distribution(data["in1"], normalize=False) == data["out1"])

def test_calc_p_distribution_2():
    data = np.load("test_data/p_distributions.npz")
    assert np.allclose(calc_p_distribution(data["in2"], normalize=True), data["out2"])

def test_calc_p_distribution_3():
    data = np.load("test_data/p_distributions.npz")
    assert np.all(calc_p_distribution(data["in3"], normalize=False) == data["out3"])

def test_calc_p_distribution_4():
    data = np.load("test_data/p_distributions.npz")
    assert np.allclose(calc_p_distribution(data["in4"], normalize=True), data["out4"])

Next the distribution for every byte is plotted. The red line represents a perfectly random distribution. Therefore, the closer the distribution approaches the red line, the less bias there is in that byte. As expected, there is a large bias in the first byte compared to the other bytes (if this is not the case ask one of the trainers to review your implementation).

In [ ]:
hists = calc_p_distribution(ineffective_intermediates, normalize=True)

fig, axes = plt.subplots(4,4, dpi=150, sharex=True, sharey=True)
fig.suptitle('Distribution of $p$ per byte index')
fig.text(0.5, 0.04, 'Byte value', ha='center')
fig.text(0, 0.5, '$p$ distribution', va='center', rotation='vertical')

for byte_index in range(16):
    ax = axes[byte_index%4][byte_index//4]
    ax.set_title(f"Byte {byte_index}", fontsize=8, y=0.75)
    ax.plot(hists[byte_index])
    ax.axhline(1/256, color = 'red')

## Exercise 1c: Calculate SEI score for every key and every byte


The SEI score is a single number that tells you how much difference exists between your observed distribution and the distribution you would expect if there was no bias at all. Therefore, the higher the SEI score the more biased our data is. If the correct key has a greater bias than an incorrect key in certain byte this property can be exploited to brute force for of the key bytes with a search space of only $2^{32}$. 

As shown in [*"SIFA: Exploiting Ineffective Fault Inductions on Symmetric Cryptography"* by Dobraunig et al.](https://eprint.iacr.org/2018/071.pdf) the SEI score can be defined as:

$
S(\hat{p})=\operatorname{SEI}(\hat{p}):=\sum_{x \in \mathcal{X}}(\hat{p}(x)-\theta(x))^2=\left(N \cdot 2^b\right)^{-1} \cdot \operatorname{CHI}(\hat{p}) .
$

### **Assignment**
**Implement an function `calc_sei_score(p_distribution)` that takes an array of p distributions, shape (16, 256), and returns the sei for every byte, shape (16,).**

In [ ]:
# Remove the following line and implement the function below
raise NotImplementedError("Complete the assignment: implement the calc_p_distribution function")

def calc_sei_score(p_distribution):
    sei_scores = np.zeros(16)

    #    NOTE p_distribution shape is (16,256)
    # 1. TODO Calculate the SEI score for each byte index
    # 2. TODO Return the SEI score for each byte index as a numpy array of 
    #         shape (16,)
    
    return sei_scores

Now run the following code to test your implementation, if all three tests pass you can continue to the next step:

In [ ]:
%%ipytest

# Test calc_sei_score function

def test_calc_sei_score_1():
    data = np.load("test_data/sei_score.npz")
    assert np.allclose(calc_sei_score(data["in1"]), data["out1"])

def test_calc_sei_score_2():
    data = np.load("test_data/sei_score.npz")
    assert np.allclose(calc_sei_score(data["in2"]), data["out2"])

def test_calc_sei_score_3():
    data = np.load("test_data/sei_score.npz")
    assert np.allclose(calc_sei_score(data["in3"]), data["out3"])

## Calculate the SEI for correct and incorrect keys

Now the SEI score can be determined for every byte for the correct key and 10 incorrect keys. We can perform both SIFA and SEFA and compare the results.

### SIFA SEI score per byte
As expected, the SEI score is in the first byte is higher for the correct key (if this is not the case ask one of the trainers to review your implementation). This means that using these ciphertexts, 4 key bytes can be bruteforced with a search space of only $2^{32}$.

In [ ]:
# Keyarray with 10 incorrect keys and 1 correct key
keys = np.array([correct_key] + [secrets.token_bytes(16) for _ in range(10)],
                dtype="|S16")

# Calculate the intermediate values for all keys and ciphertexts
intermediates = np.zeros((keys.shape[0], ineffective_ciphertexts.shape[0]), 
                         dtype="|S16")
for i, key in tqdm(enumerate(keys), total=keys.shape[0]):
    intermediates[i] = np.vectorize(aes_decrypt_to_intermediate)(key, ineffective_ciphertexts)

# Calculate the p distribution for all keys and intermediate values
hists = np.apply_along_axis(calc_p_distribution, axis=1, arr=intermediates, normalize=True)

# Calculate the SEI score for all keys and p distributions
sei_scores = np.zeros((keys.shape[0], 16))
for i, key in tqdm(enumerate(keys), total=keys.shape[0]):
    sei_scores[i] = calc_sei_score(hists[i])

# Plot the SEI score for all keys
fig, axes = plt.subplots(4,4, dpi=150, sharex=True, sharey=True)
fig.suptitle('SIFA: SEI score of all states per byte index')
fig.text(0.5, 0.04, 'Keys', ha='center')
fig.text(0, 0.5, 'SEI score', va='center', rotation='vertical')

for byte_index in range(16):
    ax = axes[byte_index%4][byte_index//4]
    ax.set_title(f"Byte {byte_index}", fontsize=8, y=0.75)
    color = ["red"] + ["blue"] * (sei_scores.shape[0]-1)
    ax.bar(np.arange(sei_scores.shape[0]), sei_scores[:, byte_index], color=color)

# Add legend
leg = fig.legend(["Correct key", "Random key"], bbox_to_anchor=(0.95, 0.5), loc='center left', borderaxespad=0)
leg.legend_handles[0].set_color('red')
leg.legend_handles[1].set_color('blue')

### SEFA SEI score per byte
As expected, the SEI score is in the first byte is higher for the correct key (if this is not the case ask one of the trainers to review your implementation). This means that using these ciphertexts, 4 key bytes can be bruteforced with a search space of only $2^{32}$.

Note that the SEI score for the correct key is much higher using SEFA than using SIFA, **why do you think this is the case?**

In [ ]:
# Keyarray with correct key and 10 random keys
keys = np.array([correct_key] + [secrets.token_bytes(16) for _ in range(10)], dtype="|S16")

# Calculate the intermediate states for all keys and ciphertexts
intermediates = np.zeros((keys.shape[0], effective_ciphertexts.shape[0]), dtype="|S16")
for i, key in tqdm(enumerate(keys), total=keys.shape[0]):
    intermediates[i] = np.vectorize(aes_decrypt_to_intermediate)(key, effective_ciphertexts)

# Calculate the p distribution for all keys and intermediate values
hists = np.apply_along_axis(calc_p_distribution, axis=1, arr=intermediates, normalize=True)

# Calculate the SEI score for all keys and p distributions
sei_scores = np.zeros((keys.shape[0], 16))
for i, key in tqdm(enumerate(keys), total=keys.shape[0]):
    sei_scores[i] = calc_sei_score(hists[i])

# Plot the SEI scores
fig, axes = plt.subplots(4,4, dpi=150, sharex=True, sharey=True)
fig.suptitle('SEFA: SEI score of all states per byte index')
fig.text(0.5, 0.04, 'Keys', ha='center')
fig.text(0, 0.5, 'SEI score', va='center', rotation='vertical')

for byte_index in range(16):
    ax = axes[byte_index%4][byte_index//4]
    ax.set_title(f"Byte {byte_index}", fontsize=8, y=0.75)
    color = ["red"] + ["blue"] * (sei_scores.shape[0]-1)
    ax.bar(np.arange(sei_scores.shape[0]), sei_scores[:, byte_index], color=color)

# Add legend
leg = fig.legend(["Correct key", "Random key"], bbox_to_anchor=(0.95, 0.5), loc='center left', borderaxespad=0)
leg.legend_handles[0].set_color('red')
leg.legend_handles[1].set_color('blue')

## Good job!

![](https://media.tenor.com/kxzr3-r6XoIAAAAM/lets-get-this-party-started-yeah.gif)

### **Now you can continue to [Exercise 2](../Exercise_2/Exercise_2.ipynb).**